In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import numpy as np
import k3d
import matplotlib.pyplot as plt
from sloppy.optic import *
from sloppy.raytracing import *
from sloppy.abcd import *
from sloppy.utils import *
from sloppy.tools import *

In [3]:
from cavities import *

In [34]:
waists_vs_param(LensCav, 'lens_dist', 1e-2)

res = degeneracy_length(LensCav, 'lens_dist', 1e-2)
Ldeg = res.x
print(res)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

     fun: 5.039003982592661e-05
 message: 'Solution found.'
    nfev: 8
  status: 0
 success: True
       x: 21.524501032332


In [39]:
(system.Ltot - Ldeg - 8.)/2.

91.89525074632989

In [38]:
Ldeg/2.

10.762250398050876

In [40]:
Ldeg

21.52450079610175

In [5]:
elements = LensCav(lens_dist=Ldeg)
sys = RaySystem(elements)

In [6]:
system = sys.abcd
print(system.Ltot)
system.waist_at(0)

213.31500228876155


array([0.01311271, 0.01311271])

In [7]:
mu = np.array([0., 0., 0., 0.])
ray = sys.screen.eigenvectors_to_rays(mu)

In [8]:
try:
    plot.close()
except:
    pass
plot = k3d.plot(camera_auto_fit=True, antialias=True)

for i, el in enumerate(sys.elements):
    plot += el.plot(opacity=0.4)

for i, el in enumerate(sys.elements):
    plot_element_ax(el, plot, length=5.)

plot_rays(ray, plot)
plot.display()

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [9]:
for el in sys.elements:
    print(el, "\n", el.Rbasis)

 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
 [[-0.26389035 -0.96455268  0.          0.        ]
 [-0.96455268  0.26389035  0.          0.        ]
 [ 0.          0.         -0.26389035 -0.96455268]
 [ 0.          0.         -0.96455268  0.26389035]]
 [[ 0.3958434   0.91831803  0.          0.        ]
 [ 0.91831803 -0.3958434   0.          0.        ]
 [ 0.          0.          0.3958434   0.91831803]
 [ 0.          0.          0.91831803 -0.3958434 ]]
 [[-0.26389035 -0.96455268  0.          0.        ]
 [-0.96455268  0.26389035  0.          0.        ]
 [ 0.          0.         -0.26389035 -0.96455268]
 [ 0.          0.         -0.96455268  0.26389035]]
 [[ 0.31257812  0.94989206  0.          0.        ]
 [ 0.94989206 -0.31257812  0.          0.        ]
 [ 0.          0.          0.31257812  0.94989206]
 [ 0.          0.          0.94989206 -0.31257812]]
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0

In [41]:
x = np.linspace(0, system.Ltot, 200)
ws = system.compute_waists(x)

plt.figure()
plt.plot(x, ws)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
%matplotlib widget
from ipywidgets import *

In [29]:
fig, (ax, bx, cx) = plt.subplots(ncols=3, figsize=(11,3.3), sharex=True, sharey=True)
ms = 8.
linea = ax.scatter([], [], c=[], cmap='jet', s=ms)
linec = cx.scatter([], [], c=[], cmap='jet', s=ms)
lineb = bx.scatter([], [], c=[], cmap='jet', s=ms)
ax.set_title('Raytracing')
bx.set_title('ABCD')
cx.set_title('ABCD (Extracted)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
 
cavfct = LensCav
parname = 'lens_dist'
res = degeneracy_length(cavfct, parname, 1e-3)
Ldeg = res.x
#print(res)

def makeidx(hit_m):
    return np.arange(hit_m.shape[0])

def update(dl=0., ar=0.0, br=0.0, ap=0., bp=0., Nrt=500, **kwargs):
        elements = cavfct(**{parname: Ldeg+dl*1e-1})
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = system.waist_at(0)[0]
        rmu1 = np.linalg.norm(np.real(mu1[:2]))
        rmu2 = np.linalg.norm(np.real(mu2[:2]))
        mu = np.real(ar*waist/rmu1*mu1*np.exp(1j*ap) + br*waist/rmu2*mu2*np.exp(1j*bp))
        
        #mu = np.real(ar*np.exp(1j*ap)*mu1 + br*np.exp(1j*bp)*mu2)
        ray0 = sys.screen.eigenvectors_to_rays(mu)
 
        traj_hit = sys.propagate(ray0, Nrt=Nrt, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])

        linea.set_offsets(hit_scr[:,:])
        linea.set_array(makeidx(hit_scr))
        
        hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lineb.set_offsets(hit_m[:,0:2])
        lineb.set_array(makeidx(hit_m))
        
        abcd_fd = sys.extract_ABCD(epsr=1e-3, epss=1e-3, Nrt=1) 
        hit_fd = propagate_ABCD(mu, abcd_fd, Nrt=Nrt)
        linec.set_offsets(hit_fd[:,0:2])
        linec.set_array(makeidx(hit_fd))
        
        linea.set_clim(vmin=0, vmax=Nrt)
        lineb.set_clim(vmin=0, vmax=Nrt)
        linec.set_clim(vmin=0, vmax=Nrt)
            
        #ax.relim(visible_only=True)
        #ax.autoscale_view(tight=True)
        ax.set_xlim(1.5*np.nanmin(hit_scr[:,0]), 1.5*np.nanmax(hit_scr[:,0]))
        ax.set_ylim(1.5*np.nanmin(hit_scr[:,1]), 1.5*np.nanmax(hit_scr[:,1]))
        fig.canvas.draw_idle()
        
lo = Layout(width='80%', height='30px')
dls = FloatSlider(value=0., min=-10., max=1., step=1e-3, readout_format='.3f', layout=lo)
ars = FloatSlider(value=0., min=0., max=6., step=1e-2, readout_format='.3f', layout=lo)
brs = FloatSlider(value=1e-3, min=0., max=6., step=1e-2, readout_format='.3f', layout=lo)
aps = FloatSlider(value=0., min=0., max=1*np.pi, step=1e-2, readout_format='.3f', layout=lo)
bps = FloatSlider(value=0., min=0., max=1*np.pi, step=1e-2, readout_format='.3f', layout=lo)
Nrts = IntSlider(value=100, min=100, max=2000, step=100, layout=lo)
#interactive(update, dl=(-10, 1, 1e-3), ar = (0,0.1,1e-3), br = (0,0.5,1e-3), ap= (0,2*np.pi,1e-2), bp= (0,2*np.pi,1e-2), Nrt=(0,2000,100))
interactive(update, dl=dls, ar = ars, br = brs, ap= aps, bp= bps, Nrt=Nrts)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='dl', layout=Layout(height='30px', width='80%'), max=…

In [35]:
fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(11,6.6), sharex='row', sharey='row')
ms = 8.
lines = [a.scatter([], [], c=[], cmap='jet', s=ms) for a in ax.flatten()]
ax[0,0].set_title('Raytracing')
ax[0,1].set_title('ABCD')
ax[0,2].set_title('ABCD (Extracted)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
 
cavfct = LensCav
parname = 'lens_dist'
res = degeneracy_length(cavfct, parname, 1e-3)
Ldeg = res.x
#print(res)

def makeidx(hit_m):
    return np.arange(hit_m.shape[0])

def update(dl=0., ar=0.0, br=0.0, ap=0., bp=0., Nrt=500, **kwargs):
        elements = cavfct(**{parname: Ldeg+dl*1e-1})
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = system.waist_at(0)[0]
        rmu1 = np.linalg.norm(np.real(mu1[:2]))
        rmu2 = np.linalg.norm(np.real(mu2[:2]))
        mu = np.real(ar*waist/rmu1*mu1*np.exp(1j*ap) + br*waist/rmu2*mu2*np.exp(1j*bp))
        
        #mu = np.real(ar*np.exp(1j*ap)*mu1 + br*np.exp(1j*bp)*mu2)
        ray0 = sys.screen.eigenvectors_to_rays(mu)
 
        traj_hit = sys.propagate(ray0, Nrt=Nrt, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
        hit_scrs = sys.screen.s_to_screen_coords(traj_hit[:,1,0,:])

        lines[0].set_offsets(hit_scr[:,:])
        lines[0].set_array(makeidx(hit_scr))
        lines[3].set_offsets(hit_scrs[:,:])
        lines[3].set_array(makeidx(hit_scrs))
        
        hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lines[1].set_offsets(hit_m[:,0:2])
        lines[1].set_array(makeidx(hit_m))
        lines[4].set_offsets(hit_m[:,2:])
        lines[4].set_array(makeidx(hit_m))
        
        abcd_fd = sys.extract_ABCD(epsr=1e-3, epss=1e-3, Nrt=1) 
        hit_fd = propagate_ABCD(mu, abcd_fd, Nrt=Nrt)
        lines[2].set_offsets(hit_fd[:,0:2])
        lines[2].set_array(makeidx(hit_fd))
        lines[5].set_offsets(hit_fd[:,2:])
        lines[5].set_array(makeidx(hit_fd))
        
        for l in lines:
            l.set_clim(vmin=0, vmax=Nrt)
            
        ax[0,0].set_xlim(1.5*np.nanmin(hit_scr[:,0]), 1.5*np.nanmax(hit_scr[:,0]))
        ax[0,0].set_ylim(1.5*np.nanmin(hit_scr[:,1]), 1.5*np.nanmax(hit_scr[:,1]))
        ax[1,0].set_xlim(1.5*np.nanmin(hit_scrs[:,0]), 1.5*np.nanmax(hit_scrs[:,0]))
        ax[1,0].set_ylim(1.5*np.nanmin(hit_scrs[:,1]), 1.5*np.nanmax(hit_scrs[:,1]))
        fig.canvas.draw_idle()
        
lo = Layout(width='80%', height='30px')
dls = FloatSlider(value=0., min=-10., max=1., step=1e-3, readout_format='.3f', layout=lo)
ars = FloatSlider(value=0., min=0., max=6., step=1e-2, readout_format='.3f', layout=lo)
brs = FloatSlider(value=1e-3, min=0., max=6., step=1e-2, readout_format='.3f', layout=lo)
aps = FloatSlider(value=0., min=0., max=1*np.pi, step=1e-2, readout_format='.3f', layout=lo)
bps = FloatSlider(value=0., min=0., max=1*np.pi, step=1e-2, readout_format='.3f', layout=lo)
Nrts = IntSlider(value=100, min=100, max=2000, step=100, layout=lo)
#interactive(update, dl=(-10, 1, 1e-3), ar = (0,0.1,1e-3), br = (0,0.5,1e-3), ap= (0,2*np.pi,1e-2), bp= (0,2*np.pi,1e-2), Nrt=(0,2000,100))
interactive(update, dl=dls, ar = ars, br = brs, ap= aps, bp= bps, Nrt=Nrts)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='dl', layout=Layout(height='30px', width='80%'), max=…

In [14]:
cavfct = LensCav
parname = 'lens_dist'
res = degeneracy_length(cavfct, parname, 1e-3)
Ldeg = res.x
print(res)
elements = cavfct(**{parname: Ldeg})
sys = RaySystem(elements)
system = sys.abcd
mu1, mu2 = system.q

     fun: 0.014627313512694156
 message: 'Solution found.'
    nfev: 6
  status: 0
 success: True
       x: 21.52450079610175


In [15]:
br = 0.01
mu = np.real(br*mu2)
ray0 = sys.screen.eigenvectors_to_rays(mu)

traj_hit = sys.propagate(ray0, Nrt=30, at_screen=True)
hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])

In [16]:
mu

array([ 0.00416093,  0.00416093, -0.00600828,  0.00600828])

In [17]:
plt.figure()
hit_1 = hit_scr[::3,:]
plt.scatter(hit_1[:,0], hit_1[:,1], c=makeidx(hit_1), cmap='jet')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
np.var(hit_1, axis=0).sum()

1.2176924669134394e-08

In [19]:
rs = np.linspace(0,0.2, 50)
distsin = np.zeros_like(rs)
devs = np.zeros_like(rs)
for i, r in enumerate(rs):
    mu = np.real(r*mu2)
    distsin[i] = np.linalg.norm(mu[:2])
    ray0 = sys.screen.eigenvectors_to_rays(mu)
    traj_hit = sys.propagate(ray0, Nrt=30, at_screen=True)
    hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
    hit_1 = hit_scr[::3,:]
    devs[i] = np.var(hit_1, axis=0).sum()

In [20]:
plt.figure()
plt.plot(distsin, devs)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
from scipy.optimize import minimize_scalar

In [22]:
def degeneracy_length_ray(cavfct, parname, r=0, scanrange=1e-3, La=None):
    def get_dev(l, r):
        elements = cavfct(**{parname: l})
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = system.waist_at(0)[0]
        rmu = np.linalg.norm(np.real(mu2[:2]))
        mu = np.real(r*waist/rmu*mu2)#np.real(r*mu2)
        ray0 = sys.screen.eigenvectors_to_rays(mu)
        traj_hit = sys.propagate(ray0, Nrt=30, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
        hit_1 = hit_scr[::3,:]
        return np.var(hit_1, axis=0).sum()
    if La is None:
        La = inspect.signature(cavfct).parameters[parname].default
    res = minimize_scalar(get_dev, bounds=((1-scanrange)*La, (1+scanrange)*La), args=(r,), method='bounded')
    return res

In [23]:
degeneracy_length_ray(cavfct, parname, r=0.1, scanrange=1e-2)

     fun: 2.2848298148807987e-16
 message: 'Solution found.'
    nfev: 7
  status: 0
 success: True
       x: 21.524365279415502

In [31]:
rs = np.linspace(0,8., 100)
Ls = np.zeros_like(rs)
lcur = Ldeg
for i, r in enumerate(rs):
    Ls[i] = degeneracy_length_ray(cavfct, parname, r=r, scanrange=1e-2, La=lcur).x
    lcur = Ls[i]

c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\abcd.py:180: RuntimeWarning: invalid value encountered in sqrt
  ws = np.sqrt(lam/(-np.pi*di.imag*n))


In [32]:
idx = np.where(rs>5.)[0][0] #only for up to
coef = np.polyfit(rs[:idx], Ls[:idx], deg=4)
coef

array([ 1.51753621e-03, -1.69461616e-02,  4.98016531e-02, -9.06637794e-02,
        2.15623193e+01])

In [33]:
plt.figure()
plt.plot(rs, Ls)
plt.title('Degeneracy length vs radius')
plt.xlabel(r'$r_{in}$ [waists]')
plt.ylabel(r'$L_{deg}$ [mm]')
plt.axhline(Ldeg, color='grey')
plt.plot(rs, np.polyval(coef, rs), '--')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
def plot_config(l=0., ar=0.0, br=0.0, ap=0., bp=0., Nrt=500, ms = 8., **kwargs):
        fig, (ax, bx, cx) = plt.subplots(ncols=3, figsize=(11,3.3), sharex=True, sharey=True)
        linea = ax.scatter([], [], c=[], cmap='jet', s=ms)
        linec = cx.scatter([], [], c=[], cmap='jet', s=ms)
        lineb = bx.scatter([], [], c=[], cmap='jet', s=ms)
        ax.set_title('Raytracing')
        bx.set_title('ABCD')
        cx.set_title('ABCD (Extracted)')
        plt.xlabel('x [mm]')
        plt.ylabel('y [mm]')
        elements = cavfct(**{parname: l})
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = system.waist_at(0)[0]
        rmu1 = np.linalg.norm(np.real(mu1[:2]))
        rmu2 = np.linalg.norm(np.real(mu2[:2]))
        mu = np.real(ar*waist/rmu1*mu1*np.exp(1j*ap) + br*waist/rmu2*mu2*np.exp(1j*bp))
        ray0 = sys.screen.eigenvectors_to_rays(mu)
 
        traj_hit = sys.propagate(ray0, Nrt=Nrt, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])

        linea.set_offsets(hit_scr[:,:])
        linea.set_array(makeidx(hit_scr))
        
        hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lineb.set_offsets(hit_m[:,0:2])
        lineb.set_array(makeidx(hit_m))
        
        abcd_fd = sys.extract_ABCD(epsr=1e-4, epss=1e-4, Nrt=1) 
        hit_fd = propagate_ABCD(mu, abcd_fd, Nrt=Nrt)
        linec.set_offsets(hit_fd[:,0:2])
        linec.set_array(makeidx(hit_fd))
        
        linea.set_clim(vmin=0, vmax=Nrt)
        lineb.set_clim(vmin=0, vmax=Nrt)
        linec.set_clim(vmin=0, vmax=Nrt)
            
        ax.set_xlim(1.5*np.nanmin(hit_scr[:,0]), 1.5*np.nanmax(hit_scr[:,0]))
        ax.set_ylim(1.5*np.nanmin(hit_scr[:,1]), 1.5*np.nanmax(hit_scr[:,1]))
        return fig

In [30]:
rw = 3.5
interp=True
if interp:
    lw = np.polyval(coef, rw)
else:
    idx = np.where(rs>rw)[0][0]
    lw = Ls[idx]
    rw = rs[idx]
f = plot_config(l=lw, br=rw, Nrt=500, ms=4)
f.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [117]:
plt.close("all")

## Eigenray

In [65]:
def plot_config_ray(mu, l=0., Nrt=500, ms = 8., **kwargs):
        fig, (ax, bx, cx) = plt.subplots(ncols=3, figsize=(11,3.3), sharex=True, sharey=True)
        linea = ax.scatter([], [], c=[], cmap='jet', s=ms)
        linec = cx.scatter([], [], c=[], cmap='jet', s=ms)
        lineb = bx.scatter([], [], c=[], cmap='jet', s=ms)
        ax.set_title('Raytracing')
        bx.set_title('ABCD')
        cx.set_title('ABCD (Extracted)')
        plt.xlabel('x [mm]')
        plt.ylabel('y [mm]')
        elements = cavfct(**{parname: l})
        sys = RaySystem(elements)
        ray0 = sys.screen.eigenvectors_to_rays(mu)

        traj_hit = sys.propagate(ray0, Nrt=Nrt, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])

        linea.set_offsets(hit_scr[:,:])
        linea.set_array(makeidx(hit_scr))
        
        hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lineb.set_offsets(hit_m[:,0:2])
        lineb.set_array(makeidx(hit_m))
        
        abcd_fd = sys.extract_ABCD(epsr=1e-4, epss=1e-4, Nrt=1) 
        hit_fd = propagate_ABCD(mu, abcd_fd, Nrt=Nrt)
        linec.set_offsets(hit_fd[:,0:2])
        linec.set_array(makeidx(hit_fd))
        
        linea.set_clim(vmin=0, vmax=Nrt)
        lineb.set_clim(vmin=0, vmax=Nrt)
        linec.set_clim(vmin=0, vmax=Nrt)
            
        ax.set_xlim(1.5*np.nanmin(hit_scr[:,0]), 1.5*np.nanmax(hit_scr[:,0]))
        ax.set_ylim(1.5*np.nanmin(hit_scr[:,1]), 1.5*np.nanmax(hit_scr[:,1]))
        return fig

In [74]:
#mu = np.array([5e-3, 0., 0, 0])
system = sys.abcd
mu1, mu2 = system.q
waist = system.waist_at(0)[0]
rmu = np.linalg.norm(np.real(mu2[:2]))
r = 5e-3
mu = np.real(r*waist/rmu*mu2)
ray0 = sys.screen.eigenvectors_to_rays(mu)
lr = np.zeros((2, 1, 3))
#lr[1,:,:] = 0.05
lr = 0.05
#reig = sys.find_eigenray(ray0, lr=lr, Nrt=30, debug=True)
reig = sys.find_eigenray_mpe(ray0, lr=lr, Nrt=30, debug=True)
mueig = sys.screen.rays_to_eigenvectors(reig)
mueig

Finished in 49 steps, reached tol 5.473e-05
Finished in 49 steps, reached tol 2.166e-09
Finished in 49 steps, reached tol 4.613e-08
Finished in 49 steps, reached tol 3.327e-08
Finished in 49 steps, reached tol 3.617e-08


array([[ 6.07047033e-05,  7.65926464e-05, -1.10603097e-04,
        -3.29846840e-04]])

In [75]:
f = plot_config_ray(np.squeeze(mueig), Ldeg)
f.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
def degeneracy_length_eigenray(cavfct, parname, r=0, scanrange=1e-3, La=None):
    def get_dev(l, r):
        elements = cavfct(**{parname: l})
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = system.waist_at(0)[0]
        rmu = np.linalg.norm(np.real(mu2[:2]))
        mu = np.real(r*waist/rmu*mu2)
        ray0 = sys.screen.eigenvectors_to_rays(mu)
        lr = 0.05
        reig = sys.find_eigenray(ray0, lr=lr, Nrt=3, debug=False)
        #reig = sys.find_eigenray_mpe(ray0, lr=lr, Nrt=30, debug=False)
        mueig = sys.screen.rays_to_eigenvectors(reig)
        
        ray1 = sys.screen.eigenvectors_to_rays(mu)
        traj_hit = sys.propagate(ray1, Nrt=30, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
        hit_1 = hit_scr[::3,:]
        return np.var(hit_1, axis=0).sum()
    if La is None:
        La = inspect.signature(cavfct).parameters[parname].default
    res = minimize_scalar(get_dev, bounds=((1-scanrange)*La, (1+scanrange)*La), args=(r,), method='bounded')
    return res

In [82]:
degeneracy_length_eigenray(cavfct, parname, r=1e-3, scanrange=1e-3, La=Ldeg)

     fun: 2.020474060964267e-08
 message: 'Solution found.'
    nfev: 10
  status: 0
 success: True
       x: 21.506287396753475

In [83]:
rs = np.linspace(0,8., 100)
Ls = np.zeros_like(rs)
lcur = Ldeg
for i, r in enumerate(rs):
    try:
        l = degeneracy_length_eigenray(cavfct, parname, r=r, scanrange=1e-2, La=lcur).x
    except:
        Ls[i] = np.nan
    else:
        Ls[i] = l
        lcur = l

In [84]:
idx = np.where(rs>5.)[0][0] #only for up to
coef = np.polyfit(rs[:idx], Ls[:idx], deg=4)
coef

array([-2.43816029e-04,  2.60880858e-03, -2.34249747e-02,  1.36223586e-02,
        2.15193646e+01])

In [85]:
plt.figure()
plt.plot(rs, Ls)
plt.title('Degeneracy length vs radius')
plt.xlabel(r'$r_{in}$ [waists]')
plt.ylabel(r'$L_{deg}$ [mm]')
plt.axhline(Ldeg, color='grey')
plt.plot(rs, np.polyval(coef, rs), '--')
plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
rw = 5.5
interp=False
if interp:
    lw = np.polyval(coef, rw)
else:
    idx = np.where(rs>rw)[0][0]
    lw = Ls[idx]
    rw = rs[idx]
f = plot_config(l=lw, br=rw, Nrt=500, ms=4)
f.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …